---
---
# Modèles de classification
---
---

Dans ce notebook, nous avons créé des modèles de classification binaire pour déterminer si la question de l'utilisateur est liée à notre domaine ou non. Puis dans un second temps, nous avons utilisé des modèles multiclasses pour sélectionner un thème si la question est dans notre domaine. 

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Nous avons utilisé Google Colab pour réduire le temps d'exécution des modèles (avec le GPU). Nous avons donc stocké les fichiers contenant les données sur Google Drive.

---
## Importation des bibliothèques et modules
---

In [0]:
import random
import re
import pandas as pd
import numpy as np
import csv
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split,GridSearchCV,KFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import keras
import sklearn
from keras.models import Sequential
from keras import layers
import tensorflow as tf

---
## Importation des données
---

In [0]:
with open("/content/gdrive/My Drive/OpenSubtitles.de-fr.fr" , encoding='utf-8') as f :
    subtitles = f.read().split("\n")

Importation des dialogues de la base OPUS, nous avons utilisé les sous titres français allemand. 

In [0]:
scrapping = pd.read_csv("/content/gdrive/My Drive/scraping.csv", sep=';', header=0)

Importation des données que nous avons récupérées par scraping pour la composante orientée tâche. 

In [0]:
scrapping.head(10)

,theme,question,reponse
0,Livraison,Quel transporteur va me livrer ?,Le nom du transporteur vous est indiqué lors d...
1,Livraison,Colissimo : quels sont les délais et modalités...,Vous êtes livré par Colissimo sous 4 à 6 jours...
2,Livraison,Geodis : quels sont les délais et modalités d’...,Vous êtes livré par Geodis sous 4 à 6 jours ou...
3,Livraison,Agediss : quels sont les délais et modalités d...,Vous êtes livré par Agediss dans la pièce de v...
4,Livraison,Chronopost : Quels sont les délais et modalité...,Vous êtes livré par Chronopost sous 4 à 6 jour...
5,Livraison,Colissimo point de retrait : quels sont les dé...,Votre commande est livrée sous 3 à 6 jours ouv...
6,Livraison,City transport : quels sont les délais et moda...,City transport livre sous 4 à 6 jours ouvrés a...
7,Livraison,Jacky Perrenot : quels sont les délais et moda...,Vous êtes livré par Jacky Perrenot dans la piè...
8,Livraison,Guisnel : quels sont les délais et modalités d...,Vous êtes livré par Guisnel dans la pièce de v...
9,Livraison,Geodis point de retrait : quels sont les délai...,Votre commande est livrée sous 3 à 6 jours ouv...


---
## Recherche des questions dans corpus de sous-titres
---

Nous avons recherché des questions dans le corpus des sous-titres dans le but de constituer des échantillons d'apprentissage et de validation composés de questions du domaine et de questions issues de la base de sous-titre OPUS. Nous allons ensuite nous servir de ces échantillons pour entrainer des modèles de classification. 

### Création d'un corpus de mots propres aux questions

In [0]:
corpus_question = ["qu'","c'est",'comment', 'pourquoi', 'quoi', 'depuis', 'combien', 'quand', 'où', 'est ce que', 'lequel', 'laquelle', 'lesquels', 'quel', 'quelle', 'quels', 'quelles', "qu'est ce que", 'êtes', 'je', 'tu', 'il', 'elle', 'nous', 'vous', 'ils']

corpus_question est une liste qui contient des mots interrogatifs.

### Recherche de questions

In [0]:
def search_questions(data):
    questions = []
    for line in data[:3000]:
        sentence = line.lower().replace("-", " ").strip()#on met la phrase en minuscule pour le traitement, et on remplace les tirets par des espaces
        for mot in corpus_question:
            if len(sentence.split("?"))==2 and re.search(r"^"+mot, sentence) and sentence not in questions: 
            #on choisit les lignes ne comportant qu'une phrase (qu'une question)
            #on recherche les questions commençant par un mot propre aux questions classiques
            #on ne garde pas les doublons
                questions.append(sentence)
    return questions

In [0]:
questions = search_questions(subtitles)

questions est une liste dont chaque élement est une question présente dans la base de sous-titres OPUS.

In [0]:
questions = random.sample(questions, 142) 
print(questions)

['il part en égypte construire un canal qui va relier la méditerranée à la mer rouge.   un canal ?', "pourquoi tu l'as pas amenée ?", "c'est l'homme de ta vie ?", "quelquefois, on peut l'aider ?", "c'est parce que tu es un arabe, c'est ça ?", 'tu parles des gardiens ?', 'tu te sens bien ?', 'tu te souviens, ménilmontant ?', 'comment va ton fils ?', "qu'est ce qui t'est arrivé ?", "jeanne, est ce que tu veux m'épouser ?", "c'est ça. tu veux nous laisser seuls face à 3 millions d'arabes ?", 'tu me fais visiter ?', 'tu es sûr de le vouloir ?', 'comment il a trouvé le dossier, cet enfoiré ?', "pourquoi m'as tu caché tout ça ?", "qu'ont ils à voir là dedans, les oncles ?", 'pourquoi les français qui vivent ici prennent pas des prénoms arabes ?', "c'est toi ?", 'tu iras voir les enfants ?', 'comment ?', 'tu veux de la sauce ?', 'tu sais où elle est ?', 'tu lui fais confiance ?', "qu'est ce qui se passe ?", 'quoi ?', 'tu crois que tes parents donneront leur fille à un musulman ?', 'vous buvez

Nous avons sélectionné aléatoirement 143 questions parmi toutes celles trouvées dans la base OPUS. Elles vont nous permettre de créer des échantillons d'apprentissage et de validation composée pour moitié des questions issues du scraping et pour moitié de la base OPUS.

### Export de la base de questions (subtitles)

In [0]:
with open("/content/gdrive/My Drive/questions_sample.csv", 'w', encoding='utf-8', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter='\n')
    writer.writerow(questions)

---

## Classification binaire

---

### Création de la base regroupant les questions métier et non métier

In [0]:
scrapping_questions = scrapping[['theme','question']]
scrapping_questions['metier'] = 1
scrapping_questions['question'] = scrapping_questions.question.str.lower()
scrapping_questions.head(10)

,theme,question,metier
0,Livraison,quel transporteur va me livrer ?,1
1,Livraison,colissimo : quels sont les délais et modalités...,1
2,Livraison,geodis : quels sont les délais et modalités d’...,1
3,Livraison,agediss : quels sont les délais et modalités d...,1
4,Livraison,chronopost : quels sont les délais et modalité...,1
5,Livraison,colissimo point de retrait : quels sont les dé...,1
6,Livraison,city transport : quels sont les délais et moda...,1
7,Livraison,jacky perrenot : quels sont les délais et moda...,1
8,Livraison,guisnel : quels sont les délais et modalités d...,1
9,Livraison,geodis point de retrait : quels sont les délai...,1


In [0]:
X_train_scraping, X_test_scraping, y_train_scraping, y_test_scraping = train_test_split(scrapping_questions.question, scrapping_questions.metier, test_size=0.29, random_state=42, stratify = scrapping_questions.theme)

In [0]:
subtitles_questions = pd.DataFrame(questions, columns=["question"])
subtitles_questions['metier'] = 0
subtitles_questions.head(10)

,question,metier
0,il part en égypte construire un canal qui va r...,0
1,pourquoi tu l'as pas amenée ?,0
2,c'est l'homme de ta vie ?,0
3,"quelquefois, on peut l'aider ?",0
4,"c'est parce que tu es un arabe, c'est ça ?",0
5,tu parles des gardiens ?,0
6,tu te sens bien ?,0
7,"tu te souviens, ménilmontant ?",0
8,comment va ton fils ?,0
9,qu'est ce qui t'est arrivé ?,0


In [0]:
subtitles_questions = subtitles_questions.sample(n=142)
subtitles_questions.head(10)

,question,metier
28,"c'est pour ça, que tu t'habilles en djellaba ?",0
85,il y a une autre femme ?,0
121,laquelle ?,0
89,vous ne l'avez jamais revu ?,0
48,"pourquoi tu m'as abandonnée, odilon ?",0
79,tu veux des femmes ?,0
93,tu sais qu'on se bat dans les rues de paris ?,0
88,tu plaisantes ?,0
63,c'est compris ?,0
64,tu as mangé ?,0


In [0]:
X_train_subtitles, X_test_subtitles, y_train_subtitles, y_test_subtitles = train_test_split(subtitles_questions.question, subtitles_questions.metier, test_size=0.29, random_state=42)

In [0]:
X_train = pd.concat([X_train_scraping, X_train_subtitles])
X_train.head(10)

108    comment consulter la validité de mon bon d’ach...
61             je souhaite un catalogue de vos articles.
83      quels sont les moyens de paiement sur internet ?
94     j’ai besoin d’une facture suite à l’achat d’un...
59                  je suis à la recherche d'un article.
131       les produits contiennent-ils du formaldéhyde ?
23     quels sont les délais et modalités d’une livra...
45     ab custom : quelles sont les modalités d'une r...
114             comment me désabonner de la newsletter ?
20     quelles sont les modalités de livraison sur le...
Name: question, dtype: object

In [0]:
X_test = pd.concat([X_test_scraping, X_test_subtitles])
X_test.head(10)

137    que faites-vous pour réduire les impressions e...
97       comment consulter le solde de ma carte cadeau ?
37                    comment être remboursé de la tva ?
18                          comment vais-je être livré ?
101    je n'ai pas reçu mon avoir suite à une command...
14        est-il possible de me faire livrer en magasin?
81         je souhaite ajouter un article à ma commande.
139    que faites-vous pour réduire l’emballage de vo...
35     puis-je retourner en magasin un article comman...
88     j'ai passé une commande avec un crédit cetelem...
Name: question, dtype: object

In [0]:
y_train = pd.concat([y_train_scraping, y_train_subtitles])
y_test = pd.concat([y_test_scraping, y_test_subtitles])

Finalement, nous obtenons un jeu d'apprentissage train_questions composé de 200 questions (100 métiers et 100 de la base OPUS) et un jeu de test test_questions composé de 84 questions (42 métiers et 42 de la base OPUS).

## Fichier modélisation

### Vectorisation

Afin d'entrainer des modèles de classification, nous avons dans un premier temps vectoriser les jeux d'apprentissage et de validation.

In [0]:
vectorizer_bin = CountVectorizer()
vectorizer_bin.fit(X_train)

X_train = vectorizer_bin.transform(X_train)
X_test  = vectorizer_bin.transform(X_test)

In [0]:
X_train.shape, y_train.shape

((200, 478), (200,))

In [0]:
X_test.shape, y_test.shape

((84, 478), (84,))

Il nous faut stocker le vectorizer créer ci-dessus, afin de pouvoir l'appliquer sur la phrase que l'utilisateur entrera : 

In [0]:
pickle.dump(vectorizer_bin, open('/content/gdrive/My Drive/vectorizer_bin.pkl', 'wb'))

Il est nécessaire de stocker le vectoriser afin de pouvoir vectoriser la question posée par l'utilisateur en temps réel.

### Modélisation du classifieur bianire

In [0]:
#import warnings
#warnings.filterwarnings("ignore")

#### Réseaux de neurones

In [0]:
input_dim = X_train.shape[1]  # Number of features

model_bin = Sequential()
model_bin.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model_bin.add(layers.Dense(1, activation='sigmoid'))

model_bin.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model_bin.summary()
model_bin.fit(X_train, y_train,
                    epochs=100,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=10)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 10)                4790      
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 11        
Total params: 4,801
Trainable params: 4,801
Non-trainable params: 0
_________________________________________________________________
Train on 200 samples, validate on 84 samples
Epoch 1/100
200/200 [==============================] - 1s 4ms/step - loss: 0.6728 - acc: 0.5500 - val_loss: 0.6580 - val_acc: 0.5833
Epoch 2/100
200/200 [==============================] - 0s 228us/step - loss: 0.6401 - acc: 0.6650 - val_loss: 0.6333 - val_acc: 0.7381
Epoch 3/100
200/200 [==============================] - 0s 273us/step - loss: 0.6040 - acc: 0.7400 - val_loss: 0.6051 - val_acc: 0.7619
Epoch 4/100
200/200 [==============================] - 0s 241us/st

In [0]:
loss_MLP, accuracy_MLP = model_bin.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.2f}".format(accuracy_MLP))

Testing Accuracy:  0.94


#### Logistic Regression

In [0]:
dict_params = {"penalty": ['l2']}
clf = GridSearchCV(estimator=LogisticRegression(), param_grid=dict_params, cv=KFold(n_splits=5),error_score='accuracy')
clf.fit(X_train, y_train)
accuracy_LR = clf.score(X_test, y_test)
print("Testing Accuracy: {:.2f}".format(accuracy_LR))

Testing Accuracy: 0.94


#### Random forest

In [0]:
dict_params = {"n_estimators": [50, 100, 200, 500], 'max_features': ['auto', 'sqrt'], 'criterion':['entropy']}
rf = GridSearchCV(estimator=RandomForestClassifier(), param_grid=dict_params, cv=KFold(n_splits=5),error_score='accuracy')
rf.fit(X_train, y_train)
accuracy_RF = rf.score(X_test, y_test)
print("Testing Accuracy: {:.2f}".format(accuracy_RF))

Testing Accuracy: 0.92


#### SVM


In [0]:
dict_params = {"C": [0.01, 0.1, 1, 10], "kernel": ["linear", "rbf"], "gamma":["auto",1e-3, 1e-4]}
clf = GridSearchCV(estimator=SVC(), param_grid=dict_params, cv=KFold(n_splits=5),error_score='accuracy')
clf.fit(X_train, y_train)
accuracy_SVM = clf.score(X_test, y_test)
print("Testing Accuracy: {:.2f}".format(accuracy_SVM))

Testing Accuracy: 0.93


### Synthèse des modèles

In [0]:
pd.DataFrame({"MLP": [accuracy_MLP], "LR": [accuracy_LR], "RF": [accuracy_RF], "SVM": [accuracy_SVM]}, index=["Accuracy"])

,MLP,LR,RF,SVM
Accuracy,0.940476,0.940476,0.916667,0.928571


Globalement, les 4 méthodes que nous avons testées donnent des résultats assez proches : une accuracy entre 0.92 et 0.96. Au vu des scores de précision, nous choisissons le modèle MLP (réseaux de neurones) pour identifier s'il s'agit d'une question métier.

Nous sauvegardons le modèle dans un fichier pickle :

In [0]:
pickle.dump(model_bin, open('/content/gdrive/My Drive/model_bin.pkl', 'wb'))

In [0]:
model_bin.save('/content/gdrive/My Drive/model_bin.h5')

Pour le récupérer :

In [0]:
model_bin = pd.read_pickle('/content/gdrive/My Drive/model_bin.pkl')

In [0]:
model_bin =  tf.keras.models.load_model('/content/gdrive/My Drive/model_bin.h5')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


### Test du modèle

On teste le modèle avec une question qu'on considère métier :

In [0]:
quest = "Colissimo comment me faire livrer ?"
X_new  = vectorizer_bin.transform([quest])
X_new

<1x473 sparse matrix of type '<class 'numpy.int64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [0]:
pd.DataFrame(model_bin.predict(X_new)[0],columns=['predict'], index=[quest])

,predict
Colissimo comment me faire livrer ?,0.982299


In [0]:
model_bin.predict(X_new)[0]

array([0.98229927], dtype=float32)

On teste le modèle avec une question qu'on ne considère pas métier :

In [0]:
quest = "Comment tu t'appelles ?"
X_new  = vectorizer_bin.transform([quest])
X_new

<1x473 sparse matrix of type '<class 'numpy.int64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [0]:
pd.DataFrame(model_bin.predict(X_new)[0],columns=['predict'], index=[quest])

,predict
Comment tu t'appelles ?,0.026235


---

## Classification multiclasses

---

### Création de la base

In [0]:
scrapping.head(10)

,theme,question,reponse
0,Livraison,Quel transporteur va me livrer ?,Le nom du transporteur vous est indiqué lors d...
1,Livraison,Colissimo : quels sont les délais et modalités...,Vous êtes livré par Colissimo sous 4 à 6 jours...
2,Livraison,Geodis : quels sont les délais et modalités d’...,Vous êtes livré par Geodis sous 4 à 6 jours ou...
3,Livraison,Agediss : quels sont les délais et modalités d...,Vous êtes livré par Agediss dans la pièce de v...
4,Livraison,Chronopost : Quels sont les délais et modalité...,Vous êtes livré par Chronopost sous 4 à 6 jour...
5,Livraison,Colissimo point de retrait : quels sont les dé...,Votre commande est livrée sous 3 à 6 jours ouv...
6,Livraison,City transport : quels sont les délais et moda...,City transport livre sous 4 à 6 jours ouvrés a...
7,Livraison,Jacky Perrenot : quels sont les délais et moda...,Vous êtes livré par Jacky Perrenot dans la piè...
8,Livraison,Guisnel : quels sont les délais et modalités d...,Vous êtes livré par Guisnel dans la pièce de v...
9,Livraison,Geodis point de retrait : quels sont les délai...,Votre commande est livrée sous 3 à 6 jours ouv...


### Echantillonage du jeu "scrapping"

In [0]:
X_train, X_test, y_train, y_test = train_test_split(scrapping.question, scrapping.theme, test_size=0.33, random_state=42, stratify = scrapping.theme)

### Vectorisation

In [0]:
vectorizer_multi = CountVectorizer()
vectorizer_multi.fit(X_train)

X_train = vectorizer_multi.transform(X_train)
X_test  = vectorizer_multi.transform(X_test)
X_train

<95x271 sparse matrix of type '<class 'numpy.int64'>'
	with 864 stored elements in Compressed Sparse Row format>

Il nous faut stocker le vectorizer créer ci-dessus, afin de pouvoir l'appliquer sur la phrase que l'utilisateur entrera : 

In [0]:
pickle.dump(vectorizer_multi, open('/content/gdrive/My Drive/vectorizer_multi.pkl', 'wb'))

### Transformation de la variable catégorielle en dummies

In [0]:
y_train_dummies = pd.get_dummies(y_train).values
y_test_dummies = pd.get_dummies(y_test).values

### Modélisation du classifieur multiclasse


#### Réseau de neurones

In [0]:
input_dim = X_train.shape[1]  # Number of features

model_multi = Sequential()
model_multi.add(layers.Dense(12, input_dim=input_dim, activation='relu'))
model_multi.add(layers.Dense(7, activation='softmax'))

model_multi.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model_multi.summary()
model_multi.fit(X_train, y_train_dummies,
                    epochs=100,
                    verbose=False,
                    validation_data=(X_test, y_test_dummies),
                    batch_size=10)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 12)                3264      
_________________________________________________________________
dense_10 (Dense)             (None, 7)                 91        
Total params: 3,355
Trainable params: 3,355
Non-trainable params: 0
_________________________________________________________________


In [0]:
loss_MLP, accuracy_MLP = model_multi.evaluate(X_test, y_test_dummies, verbose=False)
print("Testing Accuracy:  {:.2f}".format(accuracy_MLP))

Testing Accuracy:  0.68


#### Regression logistique

In [0]:
classifier_log = LogisticRegression(multi_class='multinomial')
classifier_log.fit(X_train, y_train)
accuracy_LR = classifier_log.score(X_test, y_test)

print("Testing Accuracy: {:.2f}".format(accuracy_LR))

Testing Accuracy: 0.64


#### Random forest

In [0]:
classifier = RandomForestClassifier(n_estimators = 80, criterion = 'entropy', random_state = 42)
classifier.fit(X_train, y_train)
accuracy_RF = classifier.score(X_test, y_test)

print("Testing Accuracy: {:.4f}".format(accuracy_RF))

Testing Accuracy: 0.6170


#### SVM multi-class

In [0]:
classifier = LinearSVC(random_state = 42, multi_class="crammer_singer")
classifier.fit(X_train, y_train)
accuracy_SVM = classifier.score(X_test, y_test)

print("Testing Accuracy: {:.4f}".format(accuracy_SVM))

Testing Accuracy: 0.6170


### Synthèse des modèles

In [0]:
pd.DataFrame({"MLP": [accuracy_MLP], "LR": [accuracy_LR], "RF": [accuracy_RF], "SVM": [accuracy_SVM]}, index=["Accuracy"])

,MLP,LR,RF,SVM
Accuracy,0.680851,0.638298,0.617021,0.617021


Au vu des scores de précision, nous choisissons le modèle de regression logistique pour identifier le thème de la question.

Nous sauvegardons le modèle dans un fichier pickle :

In [0]:
pickle.dump(classifier_log, open('/content/gdrive/My Drive/model_multilog.pkl', 'wb'))

In [0]:
model_multi.save('/content/gdrive/My Drive/model_multi.h5')

Pour le récupérer :

In [0]:
model_multi = pd.read_pickle('/content/gdrive/My Drive/model_multi.pkl')

### Test des modèles

On teste le modèle avec une question de thème "Livraison" :

In [0]:
quest = "Comment se déroule la livraison ?"
X_new  = vectorizer_multi.transform([quest])
X_new

<1x271 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [0]:
classifier_log.predict(X_new)

array(['Livraison'], dtype=object)

On teste le modèle avec une question de thème "Commande et Paiements" :

In [0]:
quest = "Comment obtenir une facture ?"
X_new  = vectorizer_multi.transform([quest])
X_new

<1x271 sparse matrix of type '<class 'numpy.int64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [0]:
classifier_log.predict(X_new)

array(['Commande et Paiement'], dtype=object)

On teste le modèle avec une question de thème "Compte et informations personnelles" :

In [0]:
quest = "Comment retrouver mon mot de passe ?"
X_new  = vectorizer_multi.transform([quest])
X_new

<1x271 sparse matrix of type '<class 'numpy.int64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [0]:
classifier_log.predict(X_new)

array(['Mon Compte et Informations personnelles'], dtype=object)